In [ ]:
import keybert
import nltk
import datasets
import pandas as pd

In [2]:
data = datasets.load_from_disk(dataset_path='/home/tobias/CAS Machine Learning/taproject/General-Knowledge/train')


In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df['Question'] = df['Question'].apply(lambda x: 'query: '+str(x))
df['Answer'] = df['Answer'].apply(lambda x: 'passage: '+str(x))
df

In [ ]:
input_texts = []
input_texts.extend(df['Question'][:100])
input_texts.extend(df['Answer'][:100])
input_texts

In [6]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


In [ ]:
# Each input text should start with "query: " or "passage: ".
# For tasks other than retrieval, you can simply use the "query: " prefix.
#input_texts = ['query: how much protein should a female eat',
#               'query: summit define',
#               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#               "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-small-v2')
model = AutoModel.from_pretrained('intfloat/e5-small-v2')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


In [ ]:
scores